<center><h1>Multi-class compound structural classification using sklearn</center></h1>

## About our dataset for today

The objective of this exercise is to predict the structural category of a compound using some properties derived from the monoisotopic mass and the molecular formula of the compound. ~40,000 plant lipids were classified in the following eight categories, out of which ~8000 lipids have been randomly chosen for this exercise.

<img src="Lipids.jpg" height="700" width="700">

We first used Weka for classification. Although Weka also has a command line interface and is quite easy to use, its functionality, especially for multiclass classification, is limited. SkLearn has a lot more functionality than Weka, is much more powerful in its libraries, options and execution, and is more actively developed.

The overall pipeline for analysis of the lipid dataset is as follows:

* We will first perform some exploratory analysis of the data to understand the distribution of the various features. This dataset has been filtered properly already, so there's not much to edit post exploratory analysis.

* We will divide the dataset into training and test data, and set up machine learning runs with 10X cross-validation. We will build a model with a Random Forest classifier using training data and use it to predict the class of the test dataset examples

* Then, we will estimate the accuracy of the classifier using various metrics.

* Finally, we will determine which features were important for classification purposes

If at any point you wish to determine the effect of changing some parameters on the output, or want to print some internal variables, feel free to edit the code cell and re-run the cell.

## Exploratory analysis of the data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook
%matplotlib inline

import seaborn as sns

#Read in the file
s = pd.read_csv('small_input_file.tab',sep="\t")
s.head()

#How many columns does it have? 

In [ ]:
#Get some basic numerical descriptors of the various columns
#Uncomment each of these lines one at a time
print (s.info())
print (s.describe())

#REMEMBER: Categorical features CANNOT be used for machine learning unless you transform them somehow 
# into numerical features. 

#Questions:
#1) What are categorical features?
#2) Which features in this dataset can be used for machine learning?

In [ ]:
#Plotting the distributions of the various features
s.hist(bins=20,figsize=(15,15), color='#86bf91');

#What is matplotlib plotting here? How does it know what values to make histograms from?

__Exercise 1:__ 

Notice the ; at the end of the above command. Remove it and rerun code. What happens?

In [ ]:
#Next we look at correlations between the different variables. 
allcorrel=s.corr()
allcorrel.columns
#len(allcorrel.columns)
allcorrel

#Why do you need to do this analysis? What will happen if correlated variables are included for training 
#the machine learning models?

In [ ]:
#Let us first plot a basic heatmap

fig, ax = plt.subplots()
ax.imshow(allcorrel,cmap='plasma',origin='lower')
plt.show()

As you can see, this does not make good X-axis and Y-axis labels. We need to provide them separately.

In [ ]:
#For the next command, it is important to first understand what allcorrel.columns is
print (">", allcorrel.columns)
print (">>>", len(allcorrel.columns))
print (">>>>>", np.arange(len(allcorrel.columns)))

#We will use these values for plotting a better heatmap

In [ ]:
#Create empty plot variables
fig, ax = plt.subplots()

#Lets first set the ax
ax.imshow(allcorrel,cmap='plasma',origin='lower') #other color options: viridis, inferno, hot, magma, cividis etc
#https://matplotlib.org/tutorials/colors/colormaps.html

ax.set_xticks(np.arange(len(allcorrel.columns))) #This is the >>>>> above
ax.set_yticks(np.arange(len(allcorrel.columns))) #This is the >>>>> above
ax.set_xticklabels(allcorrel.columns) #X-axis labels > above
ax.set_yticklabels(allcorrel.columns) #Y-axis labels > above
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor",size=12)
plt.setp(ax.get_yticklabels(), rotation=0, ha="right", rotation_mode="anchor",size=12)
ax.set_title("Pairwise Pearsons Correlation Coefficients between features")

#Now set the fig
fig.set_size_inches(8,8)
fig.tight_layout()

Seaborn, which is a wrapper built around matplotlib libraries, solves this problem of code complexity by making the whole block above basically a 4-line code block.

In [ ]:
#Seaborn code
plt.figure(figsize=(12,12))
ax = sns.heatmap(allcorrel, cmap="plasma")
plt.xticks(rotation=45)
ax.invert_yaxis()

__POP QUIZ:__ What does the above figure show?

## Setting up and executing the machine learning models

In [ ]:
#import required modules
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
#Example Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
#First, define which columns are labels and which are features
labels=s['Category']
feats=s[s.columns[5:]] #Are these the correct columns? Go back to the table where you defined s and check.
annot=s[s.columns[0:5]]

#Additionally, uncomment each line below to print only the top 5 rows of each variable to make sure correct columns 
#were indeed pulled out
print (feats.head())
print (labels.head())

### Variation 1: Splitting data using train-test-split

Split your data into training and test sets - here we are specifying the size of the test set as 0.2 (20%) of the main dataset, which has 8040 entries. We have to specify training and test labels as well as features. Check if the split is really 80:20.

Also note that we are not doing any Kfold cross-validation. If the dataset is sufficiently large, you can train the model on the training and test it on a sufficiently large test set. This is not advisable, but is the simplest form of training that can be done. No cross-validation required.

In [ ]:
train_feats, test_feats, train_labels, test_labels  = train_test_split (feats, labels, test_size=0.2)
print (test_labels.shape)
print (train_labels.shape)
print (test_feats.shape)
print (train_feats.shape)

#Does this look ok?

In [ ]:
#Lets make a function here to train our machine learning model
#We are running a multi-class classifier here, instead of a binary classifier (yes, no)
def random_forest_classifier(features, target):
    """
    To train the random forest classifier with features and target data
    :param features: features
    :param target: labels
    :return: trained random forest classifier
    """
    clf = RandomForestClassifier(n_estimators=100)
    #estimators stands for the number of decision trees you want to implement
    #Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
    clf.fit(features, target)
    return clf

In [ ]:
#Running the randomForestClassifier
trained_model = random_forest_classifier(train_feats, train_labels)

In [ ]:
#Let's see if the trained model predicts the test features correctly. 
pred_labels=trained_model.predict(test_feats)
pred_probs=trained_model.predict_proba(test_feats)

In [ ]:
#Uncomment each line below to understand what's going on

trained_model.__dict__ #This just shows you the model, and in this case we are only using default values, so its showing up as ().
#pred_labels
#pred_probs #Probabilities of each test instance belonging to each class

### Variation 2: Using KFold cross-validation

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(5, True, 1) #An empty splitter object for 5-fold cross validation

#Setup the basic model
rf = RandomForestClassifier(n_estimators=100)

#Train the model using cross-validation
scores = cross_val_score(rf, train_feats, train_labels, scoring='accuracy', cv=kfold, n_jobs=-1, verbose=3)

In [ ]:
scores

### Variation 3: Using grid search across all RandomForest parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
#Set up the parameters for grid search
grid_param = {
    'n_estimators': [100, 200, 300],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}
#Create an empty model
rf = RandomForestClassifier(n_estimators=100)
kfold = KFold(5, True, 1)

#Create a GridSearchCV object
grid_search = GridSearchCV(estimator=rf, param_grid=grid_param, scoring='accuracy', cv=kfold, n_jobs=-1)

#Fit the model to the data
trained_model=grid_search.fit(train_feats, train_labels)

#See all properties of trained_model
print (trained_model.__dict__)

#Print best parameters
best_params = grid_search.best_params_
print (">>>>>", best_params)

OK - now we are proceeding with the best model from grid search and seeing how it behaves with the test dataset. We will:
* Make predictions on the test data
* Output the predictions to a tab-delimited file
* See if the file is correctly formatted
* Fix any errors that occur

In [ ]:
# Performance on the test set
pred_labels=trained_model.predict(test_feats)
pred_probs=trained_model.predict_proba(test_feats)

Let us output these predictions to a file

In [ ]:
#Now check how many elements are contained within pred_labels. Is it a matrix of values?
dfx = pd.concat([test_feats, pd.DataFrame(pred_labels, columns=["Predicted labels"]), 
                 pd.DataFrame(pred_probs, columns=["A","B","C","D","E","F","G","H"])], axis=1)
dfx.to_csv(path_or_buf="results.tab",index=True,sep='\t')

#Check the output. Is it correct? You should scroll down to t

In [ ]:
#Now check how many elements are contained within pred_labels. Is it a matrix of values?

dfx1 = pd.concat([test_feats, pd.DataFrame(test_labels)], axis=1, sort=False)
dfx = pd.concat([dfx1.reset_index(),
                 pd.DataFrame(pred_labels, columns=["Predicted labels"]), 
                 pd.DataFrame(pred_probs, columns=["A","B","C","D","E","F","G","H"])], axis=1)

dfx.to_csv(path_or_buf="results2.tab",index=True,sep='\t')

In [ ]:
#why did we have to use such a complicated statement? Because there are no indexes available for the pred_labels/probs
#to merge with the test_labels and test_feats datasets.

#pred_probs
#dfx1.head()

In [ ]:
dfx1['pred_labels']=pred_labels
dfx1

The problem with sklearn is it is not straightforward to export out the predictions of the model, since the indexes of the predictions (pred_labels, pred_probs) are all messed up. Hence, a recommended strategy is -- after we are satisfied with a model -- we run it against the whole dataset and identify the test examples separately. That is what we will do.

## Checking the overall performance of the model

In this section, we will determine:
1. Accuracy of the model
2. Area Under the Receiver Operating Characteristic
3. Confusion matrix - how many instances are misclassified vs. correctly classified
4. Feature importance

In [ ]:
#class_names=test_labels[unique_labels[test_labels, pred_labels]]
class_names=list(set(train_labels))
class_names

#This is the complete, unique list of labels in the training dataset

In [ ]:

len(pred_labels)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#Using this model, predict labels of test features and see how well it behaves
for i in range(0, 20): #Change this number from 10 to something else (<20) and see what happens
        print ("Original label :: {} and Predicted label :: {}".format(list(test_labels)[i], pred_labels[i]))
print("Overall accuracy:",accuracy_score(test_labels, pred_labels))

#What do you think was printed here? 
#Uncheck the following command. Run the block again and check the output as well as the above command.
#Can you tell now?

print (">>>", len(list(test_labels)))

#Confusion matrix
class_names=list(set(train_labels))
cm=confusion_matrix(test_labels, pred_labels, labels=class_names)
print (cm)

In [ ]:
#Normalized confusion matrix
#Here we are normalizing each of the value above by the total number of examples of each category in the test dataset
cm=cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]*100
cm

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
#Plotting the confusion matrix
fig, ax = plt.subplots()
im = ax.imshow(cm, interpolation='nearest', cmap="viridis")
ax.figure.colorbar(im, ax=ax)
fig.set_size_inches(7,7)
fig.tight_layout()

# We want to show all ticks...
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       # ... and label them with the respective list entries
       xticklabels=class_names, yticklabels=class_names,
       title="Multi-class confusion matrix",
       ylabel='True label',
       xlabel='Predicted label')
#ax.tick_params(direction=2)
plt.xticks(rotation=90)

### Exercise 1:

* What does the plot show?
* Which structural categories was the model able to classify properly, and where was it confused?
* Why do you think the model was confused by those categories? What is it confused with? Why? Refer to the category definitions shown above to address these questions.


## Plotting the Area Under the Receiver Operating Characteristic (AUROC) curve

AUROC curve is always in a binary context, while here we have run a multiclass classifier. 
Hence, it is important to format the AUROC calculations in a way that enables binary calculation/plotting. We will convert all labels into only two types -- Fatty Acyls and Not Fatty Acyls - and run the classifier again

In [ ]:
#Plotting the Area Under the Receiver Operating Characteristic (AUROC) curve
from sklearn.metrics import roc_curve, auc

#Get the labels and features again, just so that we don't mess up the original variables
labels2=s['Category']
feats2=s[s.columns[5:]]
train_feats2, test_feats2, train_labels2, test_labels2  = train_test_split (feats2, labels2, test_size=0.2)

for name in class_names:
    #print (name)
    if name=='Fatty_Acyls':
        pass
    else:
        train_labels2=pd.Series(train_labels2).str.replace(name,'Not_FA') #replace labels in training
        test_labels2=pd.Series(test_labels2).str.replace(name,'Not_FA') #replace labels in test

#Run Random Forest classifier on them and print the overall model score
rf = RandomForestClassifier(n_estimators=100, oob_score=True) 
trained_model2 = rf.fit(train_feats2, train_labels2)
model_scores2=rf.score(test_feats2, test_labels2)
print ("Overall model score: ", model_scores2)

In [ ]:
#Get the Area Under the Receiver Operating Characteristic (AUC-ROC or AUROC)
y_score=trained_model2.predict_proba(test_feats2)[:, 1]
fpr, tpr, _ = roc_curve(test_labels2, y_score, pos_label='Fatty_Acyls')
#Does the figure make sense? Can AUROC be lower than random expectation?

#Change pos_label to Not_FA and see how the AUROC figure changes below.

#This is a known issue with AUROC, occurring mostly in the context of multi-class classification. 
#Some wrapper packages get around this issue. Nonetheless, we can obtain correct AUROC plots by flipping the
#class that is being plotted i.e. Not_FA instead of Fatty_Acyls


roc_auc = auc(fpr, tpr)
#print (fpr, tpr, roc_auc)

In [ ]:
type(y_score)

In [ ]:
#Plot the AUROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

## Finding which features were important for binary classification of Fatty Acyls

Feature importance is calculated as a specific metric, which is different for different algorithms. See a detailed review here: https://machinelearningmastery.com/calculate-feature-importance-with-python/. In our case, rf.feature_importance_ method by default uses the Gini impurity decrease method, which calculates the drop in "impurity" of the predicted labels, if individual features were removed from the nodes of the RF decision trees.

There are some issues with this metric with certain types of datasets. Depending on your data structure, you may need to choose different feature importance measures.

In [ ]:
train_feats2.columns

In [ ]:
## Feature importance

feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = train_feats2.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

# Print the feature ranking
print (feature_importances)

# Plot the feature importances of the forest
figcolumns=feature_importances.index.values
plt.figure()
plt.title("Feature importances")
plt.bar(range(train_feats2.shape[1]), feature_importances.importance,
       color="r", align="center")
plt.ylabel("Feature importances")
plt.xticks(range(train_feats2.shape[1]), figcolumns, rotation='vertical')
plt.xlim([-1, train_feats2.shape[1]])
plt.show()

__Copy-paste the above plot into a Word File or a Powerpoint presentation__

__Exercise 2__:

* Do the feature importances make sense, based on what you know about Fatty Acyls?

## Writing the predictions to output

This is the final step in our analysis.

In [ ]:
feats=s[s.columns[5:]]
full_labels=trained_model.predict(feats)
full_probs=trained_model.predict_proba(feats)
feats['Predictions']=full_labels
feats2=pd.concat([feats, annot, pd.DataFrame(pred_probs, columns=["A","B","C","D","E","F","G","H"])], axis=1)
feats2.to_csv(path_or_buf="results3.tab",index=True,sep='\t')

## HOMEWORK - 10 points

Repeat the binary classification with Glycerophospholipids, and check which features are important. To clarify the question, you have already performed multi-class classification. You simply need to obtain AUROC and feature importances for Glycerophospholipids vs. Not_Glycerophospholipids. 

Make a single Jupyter Notebook with your name in the file name, and upload it on Canvas. Points will be given as follows:

1. How easy is it to run the code? -- 2 points
2. Is the code well-commented? -- 2 points
3. Is the AUROC graph correct? -- 2 points
4. Is the Feature Importance graph correct? -- 2 point
5. Is there an interpretation provided for feature importance -- 2 points

This exercise should take <15 minutes to complete and submit. Submit on Canvas by Wed morning 9am.

As you can see, the coding required for Python SciKit for running ML is fairly complicated, compared to the graphical interface of Weka. However, SciKit is very powerful, and once you get past the learning curve, you will be able to quickly run different algorithms using the HPC environment. SciKit also allows encoding neural networks as well as multi-label classification (not multi-class) -- where individual instances can be assigned to more than one labels -- which is not possible in Weka. 

The problem of complexity of sklearn is addressed by a different wrapper package called PyCaret that was released in Apr 2020. Everything you did above for sklearn can be accomplished by the following five lines of code. 

## Using PyCaret

In [ ]:
s.head()

In [ ]:
from pycaret import classification
numFeat = list(s.columns[5:])
base = classification.setup(s, target='Category',ignore_features=['#LMID','Name','Subcategory','Formula'], 
                            numeric_features=numFeat)

In [ ]:
base_rf = classification.create_model('rf')

In [ ]:
tuned_rf = classification.tune_model(base_rf, n_iter=5, optimize = 'AUC')

In [ ]:
classification.evaluate_model(tuned_rf)


Look out for large datasets and relevant classification questions where ML can be applied! Let us know too, so that we can use those datasets in future classes!

In the next class, we will learn a bit about Deep Learning, and apply PyCaret for solving more machine learning problems. 

Happy coding!